In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [25]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [26]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [27]:
pd.read_sql(
    """
    SELECT COUNT() FROM boston
    """,
    con,
)

,COUNT()
0,506


1. Выведите количество пустых значений по каждой из колонок (название колонки, кол-во пустых значений)

In [31]:
query = """SELECT 
    {case_query} 
    FROM BOSTON"""
case_query = 'SUM(CASE WHEN {column} IS NULL THEN 1 ELSE 0 END) AS {column}_NULL_COUNT'
query = query.format(case_query=',\n'.join((case_query.format(column=column) for column in data.columns)))
print(query)
pd.read_sql(
    query,
    con
)

SELECT 
    SUM(CASE WHEN CRIM IS NULL THEN 1 ELSE 0 END) AS CRIM_NULL_COUNT,
SUM(CASE WHEN ZN IS NULL THEN 1 ELSE 0 END) AS ZN_NULL_COUNT,
SUM(CASE WHEN INDUS IS NULL THEN 1 ELSE 0 END) AS INDUS_NULL_COUNT,
SUM(CASE WHEN CHAS IS NULL THEN 1 ELSE 0 END) AS CHAS_NULL_COUNT,
SUM(CASE WHEN NOX IS NULL THEN 1 ELSE 0 END) AS NOX_NULL_COUNT,
SUM(CASE WHEN RM IS NULL THEN 1 ELSE 0 END) AS RM_NULL_COUNT,
SUM(CASE WHEN AGE IS NULL THEN 1 ELSE 0 END) AS AGE_NULL_COUNT,
SUM(CASE WHEN DIS IS NULL THEN 1 ELSE 0 END) AS DIS_NULL_COUNT,
SUM(CASE WHEN RAD IS NULL THEN 1 ELSE 0 END) AS RAD_NULL_COUNT,
SUM(CASE WHEN TAX IS NULL THEN 1 ELSE 0 END) AS TAX_NULL_COUNT,
SUM(CASE WHEN PTRATIO IS NULL THEN 1 ELSE 0 END) AS PTRATIO_NULL_COUNT,
SUM(CASE WHEN B IS NULL THEN 1 ELSE 0 END) AS B_NULL_COUNT,
SUM(CASE WHEN LSTAT IS NULL THEN 1 ELSE 0 END) AS LSTAT_NULL_COUNT,
SUM(CASE WHEN MEDV IS NULL THEN 1 ELSE 0 END) AS MEDV_NULL_COUNT 
    FROM BOSTON


,CRIM_NULL_COUNT,ZN_NULL_COUNT,INDUS_NULL_COUNT,CHAS_NULL_COUNT,NOX_NULL_COUNT,RM_NULL_COUNT,AGE_NULL_COUNT,DIS_NULL_COUNT,RAD_NULL_COUNT,TAX_NULL_COUNT,PTRATIO_NULL_COUNT,B_NULL_COUNT,LSTAT_NULL_COUNT,MEDV_NULL_COUNT
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2. Выведите количество уникальных значений по каждой из колонок (название колонки, кол-во уникальных значений)

In [43]:
query = """
    SELECT 
    {distinct_query}
    FROM boston
"""
distinct_query = "COUNT(DISTINCT({column})) as {column}_UNIQUE_COUNT"
query = query.format(distinct_query=',\n'.join((distinct_query.format(column=column) for column in data.columns)))
print(query)
pd.read_sql(
    query,
    con,
)


    SELECT 
    COUNT(DISTINCT(CRIM)) as CRIM_UNIQUE_COUNT,
COUNT(DISTINCT(ZN)) as ZN_UNIQUE_COUNT,
COUNT(DISTINCT(INDUS)) as INDUS_UNIQUE_COUNT,
COUNT(DISTINCT(CHAS)) as CHAS_UNIQUE_COUNT,
COUNT(DISTINCT(NOX)) as NOX_UNIQUE_COUNT,
COUNT(DISTINCT(RM)) as RM_UNIQUE_COUNT,
COUNT(DISTINCT(AGE)) as AGE_UNIQUE_COUNT,
COUNT(DISTINCT(DIS)) as DIS_UNIQUE_COUNT,
COUNT(DISTINCT(RAD)) as RAD_UNIQUE_COUNT,
COUNT(DISTINCT(TAX)) as TAX_UNIQUE_COUNT,
COUNT(DISTINCT(PTRATIO)) as PTRATIO_UNIQUE_COUNT,
COUNT(DISTINCT(B)) as B_UNIQUE_COUNT,
COUNT(DISTINCT(LSTAT)) as LSTAT_UNIQUE_COUNT,
COUNT(DISTINCT(MEDV)) as MEDV_UNIQUE_COUNT
    FROM boston



,CRIM_UNIQUE_COUNT,ZN_UNIQUE_COUNT,INDUS_UNIQUE_COUNT,CHAS_UNIQUE_COUNT,NOX_UNIQUE_COUNT,RM_UNIQUE_COUNT,AGE_UNIQUE_COUNT,DIS_UNIQUE_COUNT,RAD_UNIQUE_COUNT,TAX_UNIQUE_COUNT,PTRATIO_UNIQUE_COUNT,B_UNIQUE_COUNT,LSTAT_UNIQUE_COUNT,MEDV_UNIQUE_COUNT
0,504,26,76,2,81,446,356,412,9,66,46,357,455,229


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки).
Напишите какой вывод можно сделать по данным в этих колонках

In [322]:
query = """
    WITH add_info AS (
        SELECT {column}, MIN({column}) OVER() as min, ROW_NUMBER() OVER() row_number, COUNT() OVER() median_row FROM boston
        ORDER BY {column}
    )
    SELECT {column} as median, min, {column} - min as different FROM add_info 
    WHERE row_number = median_row / 2
"""
results = []
for column in data.columns:
    result = pd.read_sql(query.format(column=column), con)
    result['column_name'] = column
    results.append(result)

df = pd.concat(results)
df[df.different == 0]

,median,min,different,column_name
0,0.0,0.0,0.0,CHAS


In [ ]:
# Можно сделать вывод о том, что больше половины квартир граничит с рекой

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [125]:
query =  """
    SELECT AVG(max_medv.RM) - AVG(min_medv.RM) as diff
    FROM 
    (
        SELECT * FROM boston
        ORDER BY MEDV
        LIMIT {limit}
    ) min_medv,
    (
        SELECT * FROM boston
        WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    ) max_medv
"""
res = []
for limit in (25, 50, 100, 200, 300):
    result = pd.read_sql(
        query.format(limit=limit),
        con,
    )
    result['limit'] = limit
    res.append(result[['limit', 'diff']])
pd.concat(res)

,limit,diff
0,25,1.736160
0,50,1.730760
0,100,1.596880
0,200,1.572295
0,300,1.511773


In [ ]:
# Среднее количество комнат в квартире зависит от цены квартиры. Чем больше стоимость, тем больше комнат.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [158]:
pd.read_sql(
    """
    WITH rank_table as (
        SELECT LSTAT, MEDV, DENSE_RANK() OVER(ORDER BY LSTAT DESC) rank FROM boston
    )
    SELECT * FROM rank_table WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    """,
    con,
)

,LSTAT,MEDV,rank
0,9.53,50.0,280
1,8.88,50.0,301
2,7.44,50.0,334
3,5.12,50.0,397
4,4.63,50.0,414
5,4.45,50.0,420
6,3.73,50.0,432
7,3.70,50.0,433
8,3.32,50.0,440
9,3.26,50.0,441


In [ ]:
# Можно сделать вывод о том, что в квартирах с максимальной стоимостью низкий процент населения с низким статусом

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [205]:
pd.read_sql(
    """
    SELECT 
        CASE WHEN CHAS = 1 THEN 'Граничит' ELSE 'Не граничит' END as CHAS,
        AVG(MEDV) as AVG_MEDV FROM boston
    GROUP BY CHAS
    """,
    con,
)

,CHAS,AVG_MEDV
0,Не граничит,22.093843
1,Граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки). Напишите какой вывод можно сделать по этим данным.

In [206]:
query = """
    SELECT * FROM 
    (
        SELECT CHAS, AVG({colname}) avg, '{colname}' colname   FROM boston
        GROUP BY CHAS
        ORDER BY avg DESC
        LIMIT 1 
    )
    WHERE CHAS = 1
"""
queries = []
for column in data.columns:
    queries.append(query.format(colname=column))

full_query = 'UNION\n'.join(queries)

pd.read_sql(
    full_query,
    con,
)

,CHAS,avg,colname
0,1.0,0.593426,NOX
1,1.0,1.000000,CHAS
2,1.0,6.519600,RM
3,1.0,12.719143,INDUS
4,1.0,28.440000,MEDV
5,1.0,77.500000,AGE
6,1.0,372.997429,B


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [294]:
pd.read_sql(
    """
    WITH PERCENTIL AS (
        SELECT INDUS, NTILE(100) OVER(ORDER BY INDUS) AS PERC_INDUS, 
               NOX, NTILE(100) OVER(ORDER BY NOX) AS PERC_NOX 
        FROM boston
    )
    SELECT PERCENTIL.PERC_INDUS, MAX(PERCENTIL.INDUS) as INDUS, MAX(pc1.NOX) AS NOX,
    MAX(PERCENTIL.INDUS) / MAX(pc1.NOX) AS DIFF FROM PERCENTIL
    LEFT JOIN PERCENTIL pc1 ON PERCENTIL.PERC_INDUS = pc1.PERC_NOX
    WHERE PERCENTIL.PERC_INDUS IN (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
    GROUP BY PERCENTIL.PERC_INDUS
    """,
    con,
)

,PERC_INDUS,INDUS,NOX,DIFF
0,10,2.97,0.4280,6.939252
1,20,4.49,0.4429,10.137729
2,30,6.06,0.4840,12.520661
3,40,7.38,0.5070,14.556213
4,50,9.69,0.5380,18.011152
5,60,13.89,0.5800,23.948276
6,70,18.10,0.6090,29.720854
7,80,18.10,0.6710,26.974665
8,90,19.58,0.7130,27.461431
9,100,27.74,0.8710,31.848450


In [ ]:
# из результатов можно увидеть, что чем доля промышленной застройки, тем быстрее повышается концентрация оксидов азота